# Section 1: Quantum Operations - Practice Questions

**Exam Weight**: 16% (~11 questions) | **Difficulty**: Medium | **Must Master**: ✅✅✅

---

## 🎯 Key Traps to Watch For:

| Trap | Wrong Assumption | Correct Understanding |
|------|------------------|----------------------|
| Z\|0⟩ | Changes to \|1⟩ | Z\|0⟩ = \|0⟩ (no change!) |
| CX direction | Control always on q0 | `cx(control, target)` - order matters! |
| CZ symmetry | Asymmetric like CX | `cz(a,b) = cz(b,a)` - fully symmetric |
| Pauli ordering | `"ZI"` = Z on q0 | RIGHT-TO-LEFT: `"ZI"` = Z⊗I = Z on q1, I on q0 |
| T gate | T² = Z | T² = S, T⁴ = Z |
| HXH | Identity | HXH = Z (X conjugation) |

> 📖 See section_1_quantum_operations/README.md for full concepts

---

## 📚 Topics Covered (from Section Notebooks):

### Single-Qubit Gates (`single_qubit_gates.ipynb`)
- **Pauli Gates**: X (NOT), Y, Z and their matrix representations
- **Hadamard Gate (H)**: Superposition creation, HZH=X identity
- **Phase Gates**: S (π/2), T (π/4), P(θ) - S²=Z, T²=S relationships
- **Rotation Gates**: RX(θ), RY(θ), RZ(θ)
- **Gate Inverses**: `inverse()` method, adjoint gates

### Multi-Qubit Gates (`multi_qubit_gates.ipynb`)
- **CNOT/CX**: Control-target ordering, Bell state creation
- **CZ**: Symmetric controlled-Z gate
- **SWAP**: Qubit exchange, decomposition to 3 CNOTs
- **Toffoli (CCX)**: AND gate, 3-qubit controlled
- **Fredkin (CSWAP)**: Controlled swap

### State Preparation (`state_preparation.ipynb`)
- **Initialize**: `qc.initialize([a, b], qubit)`
- **Reset**: `qc.reset(qubit)` - force to |0⟩
- **Barrier**: `qc.barrier()` - prevent optimization across
- **Bell States**: All 4 Bell state preparations

In [ ]:
# Setup - Run this first!
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, Pauli
from qiskit.circuit.library import HGate, SGate, TGate, ZGate, XGate
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print("✅ Setup complete!")

---
## Part 1: Single-Qubit Gates

| Gate | Matrix | Effect | Qiskit |
|------|--------|--------|--------|
| X | [[0,1],[1,0]] | Bit flip: \|0⟩↔\|1⟩ | `qc.x(q)` |
| Z | [[1,0],[0,-1]] | Phase flip: \|1⟩→-\|1⟩ | `qc.z(q)` |
| H | 1/√2[[1,1],[1,-1]] | Superposition | `qc.h(q)` |
| S | [[1,0],[0,i]] | π/2 phase, S²=Z | `qc.s(q)` |
| T | [[1,0],[0,e^(iπ/4)]] | π/4 phase, T²=S | `qc.t(q)` |

### Q1: Apply X gate and verify state flip

In [ ]:
# Your solution: Create circuit, apply X to |0⟩, verify result is |1⟩ using Statevector


In [ ]:
# Solution Q1
qc = QuantumCircuit(1)
qc.x(0)
sv = Statevector(qc)
print(f"State after X: {sv}")
print(f"Expected: [0, 1] = |1⟩")

### Q2: Create superposition with H gate

In [ ]:
# Your solution: Apply H to |0⟩, get probabilities using probabilities_dict()


In [ ]:
# Solution Q2
qc = QuantumCircuit(1)
qc.h(0)
sv = Statevector(qc)
print(f"State |+⟩: {sv}")
print(f"Probabilities: {sv.probabilities_dict()}")
print(f"Expected: 50% |0⟩, 50% |1⟩")

### Q3: Verify S² = Z

In [ ]:
# Your solution: Use Operator to show S·S = Z



In [ ]:
# Solution Q3
s_op = Operator(SGate())
s_squared = s_op.compose(s_op)
z_op = Operator(ZGate())

print("S² matrix:")
print(np.round(s_squared.data, 3))
print(f"\nS² == Z? {np.allclose(s_squared.data, z_op.data)}")

### Q4: Rotation gates RX, RY, RZ

In [ ]:
# Your solution: Show RX(π) ≡ X (up to global phase) using equiv()


In [ ]:
# Solution Q4
qc1 = QuantumCircuit(1)
qc1.x(0)

qc2 = QuantumCircuit(1)
qc2.rx(np.pi, 0)

sv1 = Statevector(qc1)
sv2 = Statevector(qc2)

print(f"X|0⟩: {sv1}")
print(f"RX(π)|0⟩: {sv2}")
print(f"Equivalent (up to global phase)? {sv1.equiv(sv2)}")

### Q4b: Phase Gate Relationships (EXAM CRITICAL!)

Verify: T² = S, S² = Z, T⁴ = Z

In [ ]:
# Solution Q4b: Phase Gate Relationships
from qiskit.circuit.library import SGate, TGate, ZGate

s_op = Operator(SGate())
t_op = Operator(TGate())
z_op = Operator(ZGate())

# T² = S
t_squared = t_op.compose(t_op)
print("T² matrix:")
print(np.round(t_squared.data, 3))
print(f"T² == S? {np.allclose(t_squared.data, s_op.data)}")

# S² = Z
s_squared = s_op.compose(s_op)
print("\nS² matrix:")
print(np.round(s_squared.data, 3))
print(f"S² == Z? {np.allclose(s_squared.data, z_op.data)}")

# T⁴ = Z
t_fourth = t_squared.compose(t_squared)
print("\nT⁴ matrix:")
print(np.round(t_fourth.data, 3))
print(f"T⁴ == Z? {np.allclose(t_fourth.data, z_op.data)}")

---
## Part 2: Multi-Qubit Gates

| Gate | Qiskit | Effect |
|------|--------|--------|
| CNOT | `qc.cx(ctrl, tgt)` | Flip target if control=1 |
| CZ | `qc.cz(q1, q2)` | Z on target if control=1 |
| SWAP | `qc.swap(q1, q2)` | Exchange qubit states |
| Toffoli | `qc.ccx(c1, c2, tgt)` | Flip if both controls=1 |

### Q5: Create Bell state |Φ+⟩

In [ ]:
# Your solution: Create (|00⟩ + |11⟩)/√2 using H and CNOT


In [ ]:
# Solution Q5
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

sv = Statevector(qc)
print(f"Bell state |Φ+⟩: {sv}")
print(f"Probabilities: {sv.probabilities_dict()}")
print(qc.draw())

### Q5b: All Four Bell States (MUST MEMORIZE!)

Create all four Bell states with their specific gate recipes:
- |Φ+⟩ = (|00⟩ + |11⟩)/√2 → H(0) + CNOT(0,1)
- |Φ-⟩ = (|00⟩ - |11⟩)/√2 → H(0) + Z(0) + CNOT(0,1)  
- |Ψ+⟩ = (|01⟩ + |10⟩)/√2 → H(0) + X(1) + CNOT(0,1)
- |Ψ-⟩ = (|01⟩ - |10⟩)/√2 → H(0) + Z(0) + X(1) + CNOT(0,1)

In [ ]:
# Solution Q5b: All Four Bell States
def create_bell_state(name):
    qc = QuantumCircuit(2)
    qc.h(0)
    if name == 'phi_minus':
        qc.z(0)
    elif name == 'psi_plus':
        qc.x(1)
    elif name == 'psi_minus':
        qc.z(0)
        qc.x(1)
    qc.cx(0, 1)
    return qc

bell_states = {
    '|Φ+⟩': 'phi_plus',
    '|Φ-⟩': 'phi_minus', 
    '|Ψ+⟩': 'psi_plus',
    '|Ψ-⟩': 'psi_minus'
}

print("="*60)
print("ALL FOUR BELL STATES - MEMORIZE FOR EXAM!")
print("="*60)
for label, name in bell_states.items():
    qc = create_bell_state(name)
    sv = Statevector(qc)
    print(f"\n{label}:")
    print(f"  State: {sv}")
    print(f"  Probs: {sv.probabilities_dict()}")
    print(f"  Circuit: {qc.draw(output='text')}")

### Q6: GHZ state (3-qubit entanglement)

In [ ]:
# Your solution: Create (|000⟩ + |111⟩)/√2


In [ ]:
# Solution Q6
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)  # or qc.cx(1, 2)

sv = Statevector(qc)
print(f"GHZ state: {sv}")
print(f"Probabilities: {sv.probabilities_dict()}")

### Q6b: CNOT Direction Trap (EXAM WARNING!)

⚠️ `qc.cx(0, 1)` ≠ `qc.cx(1, 0)` - Direction matters!

In [ ]:
# Solution Q6b: CNOT Direction Demonstration
# Prepare |10⟩ state and apply CNOT in both directions

# Case 1: CX(0, 1) - q0 controls q1
qc1 = QuantumCircuit(2)
qc1.x(0)  # Prepare |10⟩
qc1.cx(0, 1)  # Control is q0 (which is 1), so flip q1
sv1 = Statevector(qc1)
print("Input: |10⟩")
print(f"CX(0,1) → {sv1.probabilities_dict()}")
print("Control=q0=1, so target q1 FLIPS → |11⟩\n")

# Case 2: CX(1, 0) - q1 controls q0  
qc2 = QuantumCircuit(2)
qc2.x(0)  # Prepare |10⟩
qc2.cx(1, 0)  # Control is q1 (which is 0), so NO flip
sv2 = Statevector(qc2)
print(f"CX(1,0) → {sv2.probabilities_dict()}")
print("Control=q1=0, so target q0 stays same → |10⟩")

print("\n⚠️ EXAM TRAP: Always check which qubit is control!")

### Q6c: Toffoli Gate (CCX) - AND Gate

In [ ]:
# Solution Q6c: Toffoli Gate Truth Table
print("TOFFOLI (CCX) TRUTH TABLE")
print("="*50)
print("Input → Output  (Target flips ONLY if BOTH controls=1)")
print("="*50)

for bits in range(8):
    q0, q1, q2 = (bits >> 0) & 1, (bits >> 1) & 1, (bits >> 2) & 1
    qc = QuantumCircuit(3)
    if q0: qc.x(0)
    if q1: qc.x(1)
    if q2: qc.x(2)
    qc.ccx(0, 1, 2)  # CCX(control1, control2, target)
    
    sv = Statevector(qc)
    result = list(sv.probabilities_dict().keys())[0]
    marker = " ← FLIPPED!" if bits in [3, 7] else ""  # 011, 111
    print(f"|{q2}{q1}{q0}⟩ → |{result}⟩{marker}")

print("\nSyntax: qc.ccx(control1, control2, target)")
print("Classical equivalent: target = target XOR (c1 AND c2)")

### Q7: SWAP gate demonstration

In [ ]:
# Your solution: Create |01⟩, apply SWAP, verify result is |10⟩


In [ ]:
# Solution Q7
qc = QuantumCircuit(2)
qc.x(1)  # Create |01⟩ (q0=0, q1=1)
qc.swap(0, 1)

sv = Statevector(qc)
print(f"After SWAP: {sv}")
print(f"Expected: |10⟩ (states swapped)")

### Q7b: Bloch Sphere Visualization

Visualize single-qubit states on the Bloch sphere - critical for understanding gate effects!

In [ ]:
# Solution Q7b: Bloch Sphere States
# Common states on Bloch sphere
states = {
    '|0⟩ (North Pole)': [],
    '|1⟩ (South Pole)': ['x'],
    '|+⟩ (Positive X)': ['h'],
    '|-⟩ (Negative X)': ['x', 'h'],
    '|+i⟩ (Positive Y)': ['h', 's'],
    '|-i⟩ (Negative Y)': ['h', 'sdg']
}

print("Key Bloch Sphere States:")
print("="*50)
for name, gates in states.items():
    qc = QuantumCircuit(1)
    for g in gates:
        getattr(qc, g)(0)
    print(qc.draw())
    sv = Statevector(qc)
    print(f"\n{name}:")
    print(f"  State vector: {sv.data}")
    plot_bloch_multivector(sv, title=name)
    plt.show()

---
## Part 3: Statevector Class

| Method | Returns | Description |
|--------|---------|-------------|
| `Statevector(qc)` | Statevector | Get state from circuit |
| `.data` | ndarray | Raw amplitudes |
| `.probabilities_dict()` | dict | Measurement probabilities |
| `.equiv(other)` | bool | Compare ignoring global phase |

### Q8: Statevector data and probabilities

In [ ]:
# Your solution: Get .data array and probabilities for |+⟩ state


In [ ]:
# Solution Q8
qc = QuantumCircuit(1)
qc.h(0)
sv = Statevector(qc)

print(f"Statevector: {sv}")
print(f"Raw data array: {sv.data}")
print(f"Probabilities dict: {sv.probabilities_dict()}")
print(f"Probabilities array: {sv.probabilities()}")

---
## Part 4: Operator Class

| Method | Returns | Description |
|--------|---------|-------------|
| `Operator(gate)` | Operator | Get matrix from gate |
| `.data` | ndarray | Raw matrix |
| `.compose(other)` | Operator | Matrix multiplication |

### Q9: Get Hadamard matrix

In [ ]:
# Your solution: Get H gate matrix using Operator

In [ ]:
# Solution Q9
h_op = Operator(HGate())
print("Hadamard matrix:")
print(np.round(h_op.data, 3))
print(f"\nExpected: 1/√2 * [[1,1],[1,-1]]")

### Q10: Verify H² = I

In [ ]:
# Your solution: Show H·H = Identity using compose()


In [ ]:
# Solution Q10
h_op = Operator(HGate())
h_squared = h_op.compose(h_op)

print("H² matrix:")
print(np.round(h_squared.data, 3))
print(f"\nIs identity? {np.allclose(h_squared.data, np.eye(2))}")

---
## Part 5: Pauli Class

⚠️ **EXAM DISTINCTION**: `Pauli` CLASS ≠ Pauli GATES!

| Method | Returns | Description |
|--------|---------|-------------|
| `Pauli('X')` | Pauli | Create Pauli object |
| `.commutes(other)` | bool | Check if commutes |
| `.anticommutes(other)` | bool | Check if anticommutes |
| `.evolve(gate)` | Pauli | Conjugate by gate |

⚠️ **String order is RIGHT-TO-LEFT**: `Pauli('XYZ')` = Z on q0, Y on q1, X on q2

### Q11: Pauli commutation relations

In [ ]:
# Your solution: Check if X and Y commute/anticommute


In [ ]:
# Solution Q11
px = Pauli('X')
py = Pauli('Y')
pz = Pauli('Z')

print(f"X commutes with Y? {px.commutes(py)}")
print(f"X anticommutes with Y? {px.anticommutes(py)}")
print(f"X commutes with X? {px.commutes(px)}")

### Q12: Pauli evolution through gates

In [ ]:
# Your solution: Evolve Z through H gate (should give X)

In [ ]:
# Solution Q12
pz = Pauli('Z')
evolved = pz.evolve(HGate())
print(f"Z evolved through H: {evolved.to_label()}")
print(f"Expected: X (since H·Z·H† = X)")

---
## Part 6: State Preparation

| Method | Effect |
|--------|--------|
| `qc.initialize([a,b], q)` | Set qubit to a\|0⟩+b\|1⟩ |
| `qc.reset(q)` | Force qubit to \|0⟩ |
| `qc.barrier()` | Prevent optimization across |

### Q13: Initialize to custom state

In [ ]:
# Your solution: Initialize qubit to |+⟩ = (|0⟩+|1⟩)/√2

In [ ]:
# Solution Q13
qc = QuantumCircuit(1)
qc.initialize([1/np.sqrt(2), 1/np.sqrt(2)], 0)

sv = Statevector(qc)
print(f"Initialized state: {sv}")
print(f"Probabilities: {sv.probabilities_dict()}")

### Q14: Reset operation

In [ ]:
# Your solution: Apply X then reset, verify qubit returns to |0⟩

In [ ]:
# Solution Q14
qc = QuantumCircuit(1)
qc.x(0)      # Now |1⟩
qc.reset(0)  # Force back to |0⟩

sv = Statevector(qc)
print(f"After X then reset: {sv}")
print(f"Expected: |0⟩")

---
## Part 7: Circuit Properties

| Method | Returns | Description |
|--------|---------|-------------|
| `.depth()` | int | Longest path (parallel=1) |
| `.size()` | int | Total gate count |
| `.count_ops()` | dict | Gate type counts |
| `.inverse()` | QuantumCircuit | Reversed circuit |

### Q15: Depth vs Size

In [ ]:
# Your solution: Create circuit and compare depth vs size

In [ ]:
# Solution Q15
qc = QuantumCircuit(3)
qc.h([0, 1, 2])  # 3 parallel H gates = depth 1
qc.cx(0, 1)
qc.cx(1, 2)

print(qc.draw())
print(f"\nDepth: {qc.depth()} (parallel H's count as 1)")
print(f"Size: {qc.size()} (total gates)")
print(f"Ops: {qc.count_ops()}")

### Q16: Circuit inverse

In [ ]:
# Your solution: Create circuit and show its inverse

In [ ]:
# Solution Q16
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.s(1)

print("Original:")
print(qc.draw())
print("\nInverse (gates reversed, each inverted):")
print(qc.inverse().draw())

---
## ✅ Section 1 Checklist

**Single-Qubit Gates**:
- [ ] X, Y, Z (Pauli gates)
- [ ] H (Hadamard)
- [ ] S, T (phase gates), verify S²=Z, T²=S
- [ ] RX, RY, RZ (rotation gates)

**Multi-Qubit Gates**:
- [ ] CNOT (cx), CZ, SWAP
- [ ] Bell states, GHZ state
- [ ] Toffoli (ccx)

**Classes**:
- [ ] Statevector: `.data`, `.probabilities_dict()`, `.equiv()`
- [ ] Operator: `.data`, `.compose()`
- [ ] Pauli: `.commutes()`, `.anticommutes()`, `.evolve()`

**State Preparation**:
- [ ] `initialize()`, `reset()`, `barrier()`

**Circuit Properties**:
- [ ] `.depth()`, `.size()`, `.count_ops()`, `.inverse()`